<a href="https://colab.research.google.com/github/hoangdqvn/khmer-segmentation-crf/blob/main/POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pwd

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/GR\ -\ HEDSPI\ K61/Khmer/POS

/content/drive/MyDrive/GR - HEDSPI K61/Khmer/POS


In [4]:
# Character `!` in the beginning of the line indicates a shell command
!pip install sklearn_crfsuite

# import of the crf library
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

     |████████████████████████████████| 747kB 8.8MB/s 


### Downloading Data
This section shows how to download and extract data to be analyzed. We save our data in github in a compressed format. There are multiple sizes depends on which one you want to analyze. We will start with 100 articles and we run through the whole process. This makes it fast to run the experiment as we fine tune different approaches. At the end we will rerun the full 10k document.

In [5]:
data_dir = "new_data"

print("- Count the number of files:")
!ls -al {data_dir}/*_seg_200b.txt | wc -l

- Count the number of files:
101


In [6]:
!cat {data_dir}/250007_seg_200b.txt
!cat {data_dir}/250007_tag.txt

ការ ប្រកួត បាន ចាប់ ផ្តើម ជាមួយ នឹង ក្រុម ញ៉ូវហ្សេលេន ដែល ដឹក នាំ ក្រុម ដោយ ឌីនបាឃើ ដែល មាន ប្រៀប ជាង តិច តូច ។
យ៉ាង ណា ក៏ ដោយ មិន យូរ មិន ឆាប់ ក៏ ត្រូវ បាន ចាញ់ ទៅ អាលីងហ៊ី ដោយ បញ្ចប់ ក្នុង វគ្គ ទី មួយ នាំ មុខ ប្រាំ ពីរ ប្រាំ ពីរ វិនាទី ។
ជិត បញ្ចប់ វគ្គ ទី 2 ក្រុម ញ៉ូវហ្សេលេន ព្យាយាម បាន នាំ មុខ អាលីងហ៊ី ធ្វើ អោយ ឈាន មុខ 14 វិនាទី ។
យ៉ាង ណា ក៏ ដោយ នៅ ពេល ដែល អេសយូអាយ - 100 កំពុង តែ តាម អេនហ្ស៊ីអេល - 92 ក្រុម ញ៉ូវហ្សេលេន បាន ត្រូវ ពិន័យ ដោយសារ តែ ឧប្បត្តិ ហេតុ ជិះ បុក កែម ។
មក ដល់ ជិត ចប់ នៃ ការ ប្រកួត ក្រុម ញ៉ូវហ្សេលេន បាន សំរេច នាំ មុខ 70 ម៉ែត្រ បន្ទាប់ ពី ខ្យល់ បក់ ផុត អស់ ហើយ ទូក ទាំង ពីរ ក៏ បាន ទំលាក់ ដង ចែវ របស់ ពួក គេ ហើយ អាលីងហ៊ី បាន ធ្លាក់ ចូល ក្នុង រណ្តៅ ។
ក្រុម ញ៉ូវហ្សេលេន នៅ តែ ត្រូវ បំពេញ ការ ពិន័យ របស់ ខ្លួន ដោយ ត្រូវ បត់ 360 ដឺក្រេ ហើយ ពួក គេ ក៏ បាន ធ្វើ ដូច នេះ ។
បន្ទាប់ ពី បំពេញ ជុំ ហើយ អាលីងហ៊ី បាន ជិះ ទៅ ក្បែរ ពួក គេ ហើយ បាន យក ជ័យ ជំនះ ដោយ នាំ មុខ 1 វិនាទី ។
ប្រធានាធិបតី ស្វីស មីឆែលលីនកាមីរៃ បាន អប អរ សាទរ ចំពោះ ក្រុម ដោយ ផ្ញើរ នូវ ទូរសារ ថា " ពួក យើង ជា ជន ជាតិ ស្

In [7]:
import glob
import re

path = data_dir + '/*_seg_200b.txt'  #  earlier format: *_seg.txt
files=glob.glob(path)

# global variables that use through out
seg_text = []
pos_tags = []
# unique id of the article that can be matched to docId in meta.txt
doc_ids = []
for file in files:
  filenum = re.search(r'\d+_', file).group(0)
  doc_ids.append(filenum.replace("_",""))
  f=open(file, 'r')
  lines = f.readlines()
  f.close()
  seg_text.append(lines)
  
  # read orig text -- comment out (10K docs which do not have orig text)
  f=open(file.replace('_seg_200b.txt', '_tag.txt'), 'r')
  lines = f.read().strip().split("\n")
  f.close()
  pos_tags.extend(lines)
  
print("number of file:", len(files))
print("number of doc_ids:", len(doc_ids)) # can see the origial text
print("len of seg_text:", len(seg_text), ' sample data[0][0]:', repr(seg_text[0][0]))
print("len of pos_tags:", len(pos_tags), ' sample data[0][0]:', repr(pos_tags[0][0]))

number of file: 101
number of doc_ids: 101
len of seg_text: 101  sample data[0][0]: 'ទូជីន បាន មក ដល់ ចំណុច កណ្តាល នៃ ការ ធ្វើ ដំណើរ របស់ គាត់ ដែល ជា ចំណុច សំខាន់ នៃ ទឹក សមុទ្រ 111 ថ្ងៃ បន្ទាប់ ពី ថ្ងៃ ចាប់ ចេញ ដំណើរ ពី ប្រទេស ប៉េរូ នៅ ខែ មីនា ។\n'
len of pos_tags: 20106  sample data[0][0]: 'n'


### Correction list

In [8]:
# the str assume \200b wrapped around any spaces -- this can be removed during output
# the first set use 200b string that may not visible in the note book. 
# Windows 10 shows as middle dot in Colab Classic Mode.
# the bottom set us \u200b instead for easy to spot
def get_correction_list():
  fixes=[]
  #fix bad segmented data - especially proper noun
  fixes.append(['អា​ខោន​','អាខោន​']) #account 
  fixes.append(["​ឱវា​ទ","​ឱវាទ"])
  fixes.append(["នៅ​សប្តា​ហ៏​","នៅ​សប្តាហ៏​"])
  fixes.append(['ស៊ុន​ ​ចាន់​ថុល','ស៊ុន​ ​ចាន់ថុល'])
  fixes.append(['សាយ​ ​មក​រា','សាយ​ ​មករា'])
  fixes.append(['រោង​កុន','រោងកុន'])
  fixes.append(['ដេី​មជ្រៃ','ដេីមជ្រៃ'])
  fixes.append(['​ដេី​ម​','​ដេីម​'])
  fixes.append(['មន្ទី​ពេទ្យ','មន្ទីពេទ្យ'])
  fixes.append(['ជាដរាបត​ទៅ','ជា​ដរាប​តទៅ'])
  fixes.append(['ដេី​ម្បី​','ដេីម្បី​'])
  fixes.append(['ធ្វេី​ការកាត់​យក','ធ្វេីការ​កាត់​យក']) # interesting!!!!
  fixes.append(['ភូមិ​ផ្សារ​ដី​ហុយ','ភូមិ​ផ្សារដីហុយ'])
  fixes.append(['សុខ​ ​ពេញ​វុធ','សុខ​ ​ពេញវុធ'])
  fixes.append(['ប្រតិបត្តិការបង្ក្រាប','ប្រតិបត្តិការ​បង្ក្រាប'])
  fixes.append(['health​.​com​.​kh','health.com.kh'])
  fixes.append(['ពួយ​ ​ច័ន្ទ​សុគុណ','ពួយ​ ​ច័ន្ទសុគុណ'])
  fixes.append(['ភូមិ​ ​ថ្ម​ដា','ភូមិ​ ​ថ្មដា'])
  fixes.append(['សង្កាត់​ ​បឹង​កន្សែង','សង្កាត់​ ​បឹងកន្សែង']) # TODO what is orig text???
  fixes.append(['ក្រុង​ ​បាន​លុង','ក្រុង​ ​បានលុង'])
  fixes.append(['នុត​ ​សុគន្ធ​ ​ផាន់​នី','នុត​ ​សុគន្ធ​ ​ផាន់នី'])
  #fixes.append(['ព្រះរាជាណា ច្រក កម្ពុជា','ព្រះរាជាណាច្រ ក កម្ពុជា']) #bad spelling – no better seg
  fixes.append(['កាក​បាត​ក្រហម','កាកបាត​ក្រហម']) # from https://www.redcross.org.kh/ កាកបាទក្រហមកម្ពុជា
  fixes.append(['ស្រុក​ ​កំពង់ត្រា​ច','ស្រុក​ ​កំពង់ត្រាច'])
  fixes.append(['ឃុំ​ ​ថ្ម​កែវ','ឃុំ​ ​ថ្មកែវ'])
  fixes.append(['ស្រុក​ ​អង្គរ​ជ័យ','ស្រុក​ ​អង្គរជ័យ'])
  fixes.append(['កោះ​តា​កូវ ឃុំ​ឬ​ស្សី','កោះ​តាកូវ ឃុំ​ឬស្សី'])
  fixes.append(['ម៉ាក​ ​សែន​សូនីតា','ម៉ាក​ ​សែនសូនីតា'])
  fixes.append(['ជា​ ​ដេី​ម​ ​នោះ','ជា​ ​ដេីម​ ​នោះ'])
  fixes.append(['ដេី​ម្បី​ ​ជ្រាប','ដេីម្បី​ ​ជ្រាប']) #prob eiy is two chars េ ⁣ី instead of one char
  fixes.append(['ក​រករណី','ករ​ករណី'])
  fixes.append(['ទោ​ចក្រ​យានយន្ត','ទោចក្រយានយន្ត'])
  fixes.append(['ឃុំ​ ​អន្លង់​វិល','ឃុំ​ ​អន្លង់វិល'])
  fixes.append(['​ពោធ៍​សាត់','​ពោធ៍សាត់'])
  fixes.append(['ណាក់​ ​ស្រី​ណា','ណាក់​ ​ស្រីណា'])
  fixes.append(['ពិធីការ​នីមួយ','ពិធីការនី​មួយ']) #valid both ways, need context
  fixes.append(['តូរុន​តូ','តូរុនតូ']) #Toronto
  fixes.append(['ដាយ​ណូស័រ','ដាយណូស័រ']) #Dinosaur
  fixes.append(['ហ្គារ៉ាល់​ហូស','ហ្គារ៉ាល់ហូស']) #Guarulhos in Sao Paulo
  fixes.append(['ខេមរៈ​ ​សិរី​មន្ត','ខេមរៈ​ ​សិរីមន្ត'])#ខេមរៈ \u200bសិរី\u200bមន្ត
  fixes.append(['ឱម​ ​យ៉ិន​ទៀង','ឱម​ ​យ៉ិនទៀង'])
  fixes.append(['មហា​លាប','មហាលាប'])
  fixes.append(['ការចាប់អារម្ម​ណ៏​','ការចាប់អារម្មណ៏​'])
  fixes.append(['វ៉ិច​ទ័រ​','វ៉ិចទ័រ​']) #rector
  fixes.append(['សុវណ្ណ​ ​ឬ​ទ្ធី','សុវណ្ណ​ ​ឬទ្ធី'])
  fixes.append(['ខាន់​ ​ចាន់​សុផល','ខាន់​ ​ចាន់សុផល'])
  fixes.append(['ទំនួលខុសត្រូវ​','ទំនួល​ខុសត្រូវ​'])
  fixes.append(['ឯ​ណេះ','ឯណេះ']) #confirm with Chuon Nath Dict
  fixes.append(['ហេយ៍​វ៉ិន','ហេយ៍វ៉ិន']) #heaven
  fixes.append(['សុខ​ ​ពេញ​វុធ','សុខ​ ​ពេញវុធ'])
  fixes.append(['វ៉ុល​កា','វ៉ុលកា']) #vulgar
  fixes.append(['ជា​ម​ ​ហុី​ម','ជាម​ ​ហុីម'])
  fixes.append(['​ការ​ប្រកាស​','​ការប្រកាស​'])
  fixes.append(['អនុ​ប្រធាន','អនុប្រធាន'])
  fixes.append(['កិច្ចខិតខំប្រឹងប្រែង','កិច្ច​ខិតខំ​ប្រឹងប្រែង']) #???
  fixes.append(['សស្អាត','ស​ស្អាត'])
  fixes.append(['ហាន់​ ​ជី​អ៊ុន','ហាន់​ ​ជីអ៊ុន'])
  fixes.append(['ស៊ុន​ ​ចាន់​ថុល','ស៊ុន​ ​ចាន់ថុល'])
  fixes.append(['លីន​ដា​','លីនដា​'])
  fixes.append(['យានយន្តបន្ទាប់ពី','យានយន្ត​បន្ទាប់ពី'])
  fixes.append(['ព្រៃ​ស្អាក​ស្រុក​អន្លង់​វែង','ព្រៃស្អាក​ស្រុក​អន្លង់វែង'])
  fixes.append(['​ប៉ា​កុង​','​ប៉ាកុង​']) # first 50 articles
  fixes.append(['ឈើ​ ​អុស​ក្រាក់','ឈើ​ ​អុសក្រាក់'])
  fixes.append(['រតនៈ​គីរី​','រតនៈគីរី​'])
  fixes.append(['ឯក​ឧត្តម','ឯកឧត្តម'])
  fixes.append(['ឱម​ ​យិន​ទៀង','ឱម​ ​យិនទៀង'])
  fixes.append(['​ចំដែនដី​','​ចំ​ដែនដី​']) # orig has space after រៀប
  fixes.append(['ឱវា​ទ','ឱវាទ']) # correct spelling
  fixes.append(['អ៊ិន​វេស​មិន','អ៊ិនវេសមិន']) #investment
  fixes.append(['សហគម​ន៏','សហគមន៏'])
  fixes.append(['​គ្រូ​សារ​','​គ្រូសារ​']) # orig has bad ub, mispelled but should be removed before seg
  fixes.append(['សុផុនពេល​សួរ','សុផុន​ពេល​សួរ'])
  fixes.append(['ស្រុក​កែវ​សីម៉ា','ស្រុក​កែវសីម៉ា'])
  fixes.append(['ហាន់​ជ័យ','ហាន់ជ័យ'])
  fixes.append(['បូរី​ ​ស​ចនសុន','បូរីស​ ​ចនសុន'])
  fixes.append(['បូរី​ ​ស​ចន​សុន','បូរីស​ ​ចនសុន'])
  fixes.append(['បូរីស​ ​ចន​សុន','បូរីស​ ​ចនសុន'])
  fixes.append(['អំណ​រ​គុណ','អំណរ​គុណ']) # bad orig
  fixes.append(['ផលប៉ះពាល់​','ផល​ប៉ះពាល់​']) # ???
  fixes.append(['អ្នកវិនិយោគ​','អ្នក​វិនិយោគ​']) #???
  fixes.append(['ភូមិ​ ​ស្រះ​ជ្រៃ​','ភូមិ​ ​ស្រះជ្រៃ​'])
  fixes.append(['ឃុំ​ ​បន្ទាយ​ឆ្មា​','ឃុំ​ ​បន្ទាយឆ្មា​'])
  fixes.append(['ស្រុក​ ​ថ្ម​ពួក​','ស្រុក​ ​ថ្មពួក​'])
  fixes.append(['ទ្វីត​ធឺ','ទ្វីតធឺ']) #twitter
  fixes.append(['​ធំ​ដុំ','​ធំដុំ'])
  fixes.append(['​រួមមាន​','​រួម​មាន​']) #?
  fixes.append(['គុយ​វ៉ែត','គុយវ៉ែត'])
  fixes.append(['ងងុយគេង','ងងុយ​គេង'])
  fixes.append(['ស​ម​ ​រង្ស៊ី','សម​ ​រង្ស៊ី'])
  fixes.append(['សោ​ ​ចាន់​ដេត','សោ​ ​ចាន់ដេត'])
  fixes.append(['ជាលាយលក្ខណ៍អក្សរ','ជា​លាយលក្ខណ៍​អក្សរ'])
  fixes.append(['ផលិតកម្មវិធី','ផលិតកម្ម​វិធី'])
  fixes.append(['​សាជាថ្មី​','​សា​ជា​ថ្មី​']) # ???
  fixes.append(['ឧកញ៉ា​ស្រី​ ​ចាន់​ថន','ឧកញ៉ា​ ​ស្រី​ ​ចាន់ថន'])
  fixes.append(['ភូមិ​ ​ព្រៃ​ល្វា','ភូមិ​ ​ព្រៃល្វា'])
  fixes.append(['សង្កាត់​ ​ចោម​ចៅ ','សង្កាត់​ ​ចោមចៅ'])
  fixes.append(['ខណ្ឌ​ ​ពោធិ៍​សែន','ខណ្ឌ​ ​ពោធិ៍សែន'])
  fixes.append(['ឃុំ​ ​ព្រែក​តា​មាក់','ឃុំ​ ​ព្រែកតាមាក់'])
  fixes.append(['​បាត់បង្ក','​បាត់​បង្ក'])
  fixes.append(['កើតមានឡើង','កើត​មាន​ឡើង'])
  fixes.append(['លូក​លាន់','លូកលាន់'])
  fixes.append(['លោក​ ​ជិន​ ​ម៉ាលី​ន','លោក​ ​ជិន​ ​ម៉ាលីន'])
  fixes.append(['អ្នករាយការណ៍','អ្នក​រាយការណ៍'])
  fixes.append(['គង់​ ​រ៉ៃ​យ៉ា','គង់​ ​រ៉ៃយ៉ា'])
  fixes.append(['លោក​ ​សួង​ ​នាគ​ព័ន្ធ','លោក​ ​សួង​ ​នាគព័ន្ធ'])
  fixes.append(['ធ្វើទុក្ខបុកម្នេញ','ធ្វើ​ទុក្ខបុកម្នេញ'])  # doc 75 mark
  fixes.append(['បោះបង់ចោល','បោះបង់​ចោល'])
  fixes.append(['ខ្លួនឯង','ខ្លួន​ឯង']) # ???
  fixes.append(['សប្បាយចិត្ត','សប្បាយ​ចិត្ត'])
  fixes.append(['មើលទៅ','មើល​ទៅ']) # ??? if
  fixes.append(['សោយសុខ','សោយ​សុខ'])
  fixes.append(['ទៅលើ','ទៅ​លើ'])
  fixes.append(['ឃុំខ្លួន','ឃុំ​ខ្លួន'])
  fixes.append(['ជាប់ឃុំ','ជាប់​ឃុំ'])
  fixes.append(['លោក​ ​ហេង​ ​ដូន​នី','លោក​ ​ហេង​ ​ដូននី'])
  fixes.append(['លោក​ ​កែម​ ​គិម​ស្រន់','លោក​ ​កែម​ ​គិមស្រន់'])
  fixes.append(['ស្រុក​ ​វាល​វែង','ស្រុក​ ​វាលវែង'])
  fixes.append(['ចោទប្រកាន់តែ','ចោទប្រកាន់​តែ']) # doc 80 mark
  fixes.append(['ឈិញ​ ​ស៊ី​ថា','ឈិញ​ ​ស៊ីថា'])
  fixes.append(['ដាក់ពាក្យបណ្តឹង','ដាក់​ពាក្យ​បណ្តឹង'])
  fixes.append(['ម៉ាក​ ​ណូ​គា','ម៉ាក​ ​ណូគា'])
  fixes.append(['ធូ​ ​ស្រី​ទូច','ធូ​ ​ស្រីទូច'])
  fixes.append(['ម៉ូតូកង់​បី','ម៉ូតូ​កង់​បី'])
  fixes.append(['ឃុំ មង់​រៀវ','ឃុំ មង់រៀវ'])
  fixes.append(['លោក​ ​ជា​ ​ចាន់​តូ','លោក​ ​ជា​ ​ចាន់តូ'])
  fixes.append(['ស៊ែ​ស្វី​ច','ស៊ែ​ស្វីច']) #shared switch
  fixes.append(['ប្រតិបត្តិការផ្ទេរ','ប្រតិបត្តិការ​ផ្ទេរ'])
  fixes.append(['បញ្ជាការស្រាល','បញ្ជាការ​ស្រាល'])
  fixes.append(['F​-​16','F-16'])
  fixes.append(['C​-​17','C-17'])
  fixes.append(['គោលនយោបាយ','គោល​នយោបាយ']) # id 90 mark
  fixes.append(['ហាន​ ​សុខ​ន','ហាន​ ​សុខន'])
  fixes.append(['ខៀវ​ ​ទេ​ព','ខៀវ​ ​ទេព'])
  fixes.append(['កាញារី​ទ្ធ','កាញារីទ្ធ'])
  fixes.append(['ជី​ហែ','ជីហែ']) # ផ្សារជីហែ
  fixes.append(['វិទ្យាល័យ​ ​ជី​ហែរ','វិទ្យាល័យ​ ​ជីហែរ'])
  fixes.append(['ស្រុកោះ​កោះ​សូទិន','ស្រុកោះ​កោះសូទិន'])
  fixes.append(['ជំរុញឲ្យ','ជំរុញ​ឲ្យ'])
  fixes.append(['ការបាក់ទឹកចិត្ត','ការបាក់​ទឹកចិត្ត'])
  fixes.append(['ហង់ ជួន​ណារីតា','ហង់ ជួនណារីតា'])
  fixes.append(['ជាទីមោទនៈ','ជា​ទី​មោទនៈ'])
  fixes.append(['សឿ សុជា​តា','សឿ សុជាតា'])
  fixes.append(['ហង់​ ​ជួន​ណារ៉ុន','ហង់​ ​ជួនណារ៉ុន'])
  fixes.append(['ក​សាង​ធនធាន','កសាង​ធនធាន'])
  fixes.append(['ស្នង​ឬ​ស្សី','ស្នង​ឬស្សី'])
  fixes.append(['ខៀវ​ ​កាញារី​ទ្ធ','ខៀវ​ ​កាញារីទ្ធ'])
  fixes.append(['ត្រេន​ ​ដី​ង','ត្រេនដីង']) #trending
  fixes.append(['ទៅវិញទៅមក','ទៅ​វិញ​ទៅ​មក']) # ??
  fixes.append(['ក្ដី​ស្រមៃ','ក្ដីស្រមៃ']) #prob mispell jerg da
  fixes.append(['ឬ​ស្សីកែវ','ឬស្សីកែវ'])
  fixes.append(['អេង​ ​សុវណ្ណ​តារា','អេង​ ​សុវណ្ណតារា'])
  fixes.append(['កុង​ទីន​រ័','កុងទីនរ័'])
  fixes.append(['ជាបន្តបន្ទាប់','ជា​បន្តបន្ទាប់'])
  fixes.append(['ចលត័ជាប្រចាំ','ចលត័​ជា​ប្រចាំ'])
  fixes.append(['ឃុំ​ដី​ឥដ្ឋ','ឃុំ​ដីឥដ្ឋ'])
  fixes.append(['វី​ដែ​អូ','វីដែអូ']) #misspell វីដេអូ 
  fixes.append(['ត្រី​ខ​','ត្រីខ​'])
  fixes.append(['ទឹក​ត្រី​','ទឹកត្រី​'])
  fixes.append(['បង្ខំឲ្យ','បង្ខំ​ឲ្យ']) # first hundred docs
  fixes.append(['ស្ដុក\u200bប្រវឹក','ស្ដុកប្រវឹក'])
  fixes.append(['វាល\u200bរិញ\u200b','វាលរិញ\u200b'])
  fixes.append(['ណៃ\u200b \u200bវង្ស\u200bដា','ណៃ\u200b \u200bវង្សដា'])
  fixes.append(['លោក\u200b \u200bឃ\u200bន \u200bជឺ ','លោក\u200b \u200bឃន\u200b \u200bជឺ '])
  fixes.append(['បុត\u200b \u200bចាន់\u200bណា','បុត\u200b \u200bចាន់ណា'])
  fixes.append(['ឯ\u200bម\u200b \u200bច័ន្ទ\u200bមក\u200bរា','ឯម\u200b \u200bច័ន្ទមករា'])
  fixes.append(['សង់\u200bដ្រី\u200bន\u200b \u200bឌុយ\u200bរី','សង់ដ្រីន\u200b \u200bឌុយរី'])
  fixes.append(['សុខ\u200b \u200bពេញ\u200bវុធ','សុខ\u200b \u200bពេញវុធ'])
  fixes.append(['សោម \u200bពុទ្ធ\u200bតារា','សោម \u200bពុទ្ធតារា'])
  fixes.append(['ម៉ៅ\u200b ច័ន្ទ\u200bមធុរិទ្ធ','ម៉ៅ\u200b ច័ន្ទមធុរិទ្ធ'])
  fixes.append(['បូពិន្ទ\u200b ណា\u200bរ័ត្ន','បូពិន្ទ\u200b ណារ័ត្ន'])
  fixes.append(['ជា\u200b \u200bប៊ុន\u200bហេង','ជា\u200b \u200bប៊ុនហេង'])
  fixes.append(['ណារិក\u200bដ្រា\u200b \u200bមូឌី','ណារិកដ្រា\u200b \u200bមូឌី'])
  fixes.append(['ស្រុក\u200bព្រៃ\u200bនប់','ស្រុក\u200bព្រៃនប់'])
  fixes.append(['ស្រុក\u200bព្រៃ\u200bនប់','ស្រុក\u200bព្រៃនប់'])
  fixes.append(['ខណ្ឌ\u200bចំការ\u200bមន','ខណ្ឌ\u200bចំការមន'])
  fixes.append(['ខណ្ឌ\u200bពោធិ៍\u200bសែន\u200bជ័យ','ខណ្ឌ\u200bពោធិ៍សែនជ័យ'])
  fixes.append(['សង្កាត់\u200bទឹក\u200bល្អក់','សង្កាត់\u200bទឹកល្អក់'])
  fixes.append(['សង្កាត់\u200bផ្សារ\u200bថ្មី','សង្កាត់\u200bផ្សារថ្មី'])
  fixes.append(['សង្កាត់\u200bជ្រោយ\u200bចង្វារ','សង្កាត់\u200bជ្រោយចង្វារ'])
  fixes.append(['សង្កាត់\u200bវាល\u200bវង់','សង្កាត់\u200bវាលវង់'])
  fixes.append(['សង្កាត់\u200bកោះ\u200bដាច់','សង្កាត់\u200bកោះដាច់'])
  fixes.append(['សង្កាត់\u200bចាក់អង្រែ\u200bលើ','សង្កាត់\u200bចាក់អង្រែលើ'])
  fixes.append(['សង្កាត់\u200bទន្លេ\u200bបាសាក់','សង្កាត់\u200bទន្លេបាសាក់'])
  fixes.append(['ឃុំ\u200bត្រពាំង\u200bជោ','ឃុំ\u200bត្រពាំងជោ'])
  fixes.append(['ឃុំ\u200bក្រាំង\u200bល្វា','ឃុំ\u200bក្រាំងល្វា'])
  fixes.append(['ឃុំ\u200bដូន\u200bសរ','ឃុំ\u200bដូនសរ'])
  fixes.append(['ឃុំ\u200bអូរ\u200bតាប៉ោង','ឃុំ\u200bអូរតាប៉ោង'])
  fixes.append(['ឃុំ\u200bក្បាល\u200bដំរី','ឃុំ\u200bក្បាលដំរី'])
  fixes.append(['ឃុំ\u200bត្រពាំង\u200bគង','ឃុំត្រពាំងគង'])
  fixes.append(['ភូមិ\u200bច្រក\u200bទៀក','ភូមិ\u200bច្រកទៀក'])
  fixes.append(['ភូមិ\u200bខ្សាច់\u200bស','ភូមិ\u200bខ្សាច់ស'])
  fixes.append(['ភូមិ\u200bអូរ\u200bត្រូន','ភូមិ\u200bអូរត្រូន'])
  fixes.append(['ភូមិ\u200bថ្ម\u200bធំ','ភូមិ\u200bថ្មធំ'])
  fixes.append(['ភូមិ\u200bចំការ\u200bដូង','ភូមិ\u200bចំការដូង'])
  fixes.append(['ភូមិ\u200bព្រៃ\u200bផ្តៅ','ភូមិ\u200bព្រៃផ្តៅ'])
  fixes.append(['ភូមិ\u200bបឹង\u200bវែង','ភូមិ\u200bបឹងវែង'])
  fixes.append(['\u200b \u200b.\u200b \u200bcom\u200b','.com\u200b'])
  fixes.append(['ចូ\u200b \u200bបាយ\u200bដិន','ចូ\u200b \u200bបាយដិន'])
  fixes.append(['\u200bណាន់\u200bស៊ី\u200b','\u200bណាន់ស៊ី\u200b'])
  fixes.append(['លោក\u200bត្រាំតទៅទៀត','លោក\u200bត្រាំ\u200bតទៅទៀត'])

  #fixes.append(['','']) #u200bប៊ុល\u200bហ្គារី \u200b,  \u200bហុង\u200bគ្រី  Hungary
  return fixes


fixes = get_correction_list()
print("count", len(fixes), " data[0]:", fixes[0])

count 199  data[0]: ['អា\u200bខោន\u200b', 'អាខោន\u200b']


### Parse and correct sentences

In [9]:
def correct_str(str):
  for f in fixes:
    str = str.replace(f[0], f[1])
  return str

def cleanup_str(str):
  str = str.strip('\u200b').strip()
  str = str.replace("  ", " ") # clean up 2 spaces to 1
  str = str.replace(" ", "\u200b \u200b")   # ensure 200b around space
  # clean up
  str = str.replace("\u200b\u200b", '\u200b')   # clean up dupe 200b
  str = str.replace("\u200b\u200b", '\u200b')   # in case multiple
  str = correct_str(str) # assume space has 200b wrapped around
  
  # remove special characters
  str = str.replace(u"\u2028", "") # line separator
  str = str.replace(u"\u200a", "")  # hair space
  str = str.strip().replace('\n','').replace('  ',' ')
  return str

# setup training data using seg
sentences = list()
for i, text in enumerate(seg_text):
    for sentence in text:
        sentences.append(cleanup_str(sentence))
print("total len of sentences:", len(sentences))

# id=99
# print("id:",doc_ids[id])
# for i,s in enumerate(seg_text[id]):
#   print("sample sentence: %s" %(cleanup_str(s)))
#   print("orig sentence: %s" %(repr(orig_text[id][i])))


total len of sentences: 20106


### Check count and searching

In [10]:
# search more match text in the correction list
import re
total = 0
for sen in sentences[0:813]: #todo search small set only
  for f in fixes:
    m = [m.start() for m in re.finditer(f[0], sen)]
    n = [m.start() for m in re.finditer(f[1], sen)]
    if len(m) >0:
      total += len(m)
      s = sen[m[0]:(m[0]+20)]
      #if (len(m)) > 0: print(total, "len:", len(m), " str:", repr(f[0]), "sen:", sen)
      #print("match: search=", repr(f[0]), "fix:", repr(f[1]), " sen:", repr(s))
      
print("total corrections:",total, " total sen:", len(sentences)) #127 -> 147/871



total corrections: 0  total sen: 20106


## Create Features
This section implements different methods to generate KCC and features for CRF.

### KCC

We implementated the segmentation of Khmer orthographic or Khmer character cluster (KCC) in a simpler way. Instead of check for all possible rules from the start of each character, we just check the next character and determine whether it is a type that start a new KCC. So the current character is the end of the current KCC. If there are series of invalid KCC string, this approach does not separate those. But segmented it would leave the next segment invalid any way.

In addition, we take advantage of this cluster and cluster non segmentable characters like number and non-Khmer word.

In [11]:
#@title Segment into KCCs 

# list of constants needed for KCC and feature generation
# consonant and independent vowels
KHCONST = set(u'កខគឃងចឆជឈញដឋឌឍណតថទធនបផពភមយរលវឝឞសហឡអឣឤឥឦឧឨឩឪឫឬឭឮឯឰឱឲឳ')
KHVOWEL = set(u'឴឵ាិីឹឺុូួើឿៀេែៃោៅ\u17c6\u17c7\u17c8')
# subscript, diacritics
KHSUB = set(u'្')
KHDIAC = set(u"\u17c9\u17ca\u17cb\u17cc\u17cd\u17ce\u17cf\u17d0") #MUUSIKATOAN, TRIISAP, BANTOC,ROBAT,
KHSYM = set('៕។៛ៗ៚៙៘,.? ') # add space
KHNUMBER = set(u'០១២៣៤៥៦៧៨៩0123456789') # remove 0123456789
# lunar date:  U+19E0 to U+19FF ᧠...᧿
KHLUNAR = set('᧠᧡᧢᧣᧤᧥᧦᧧᧨᧩᧪᧫᧬᧭᧮᧯᧰᧱᧲᧳᧴᧵᧶᧷᧸᧹᧺᧻᧼᧽᧾᧿')

def is_khmer_char(ch):
  if (ch >= '\u1780') and (ch <= '\u17ff'): return True
  if ch in KHSYM: return True
  if ch in KHLUNAR: return True
  return False

def is_start_of_kcc(ch):
  if is_khmer_char(ch):
    if ch in KHCONST: return True
    if ch in KHSYM: return True
    if ch in KHNUMBER: return True
    if ch in KHLUNAR: return True
    return False
  return True

# kcc base - must surround space with \u200b using cleanupstr()
def seg_kcc(str_sentence):
    segs = []
    cur = ""
    sentence = str_sentence
    #for phr in str_sentence.split(): #no longer split by space, use 200b
    #    print("phr: '", phr,"'")
    for word in sentence.split('\u200b'):
      #print("PHR:[%s] len:%d" %(phr, len(phr)))
      for i,c in enumerate(word):
          #print(i," c:", c)
          cur += c
          nextchar = word[i+1] if (i+1 < len(word)) else ""
          
          # cluster non-khmer chars together
          if not is_khmer_char(c) and nextchar != " " and nextchar != "" and not is_khmer_char(nextchar): 
            continue
          # cluster number together
          if c in KHNUMBER and nextchar in KHNUMBER: 
            continue
            
          # cluster non-khmer together
          # non-khmer character has no cluster
          if not is_khmer_char(c) or nextchar==" " or nextchar=="":
              segs.append(cur)
              cur=""
          elif is_start_of_kcc(nextchar) and not (c in KHSUB):
              segs.append(cur)
              cur="" 
        # add space back after split
        #segs.append(" ")   
    return segs # [:-1] # trim last space

# testing some text
t1 = "យោងតាមប្រភពព័ត៌មានបានឱ្យដឹងថា កាលពីពេលថ្មីៗនេះក្រុមចក្រភពអង់គ្លេស Royal Marines ដែលមានមូលដ្ឋាននៅ Gibraltar បានរឹបអូសយកនាវាដឹកប្រេងឆៅរបស់អ៊ីរ៉ង់ដែលធ្វើដំណើរទៅកាន់រោងចក្រចម្រាញ់ប្រេងនៅក្នុងប្រទេសស៊ីរី ដោយក្រុងឡុងដ៍អះអាងថា ការរឹបអូសត្រូវបានគេសំដៅអនុវត្ត៕"
t2 = "ខែThis is a test. N.B. ខែ? Test?"
t3 = "នៅរសៀលថ្ងៃទី២២ ខែ កក្កដា ឆ្នាំ២០១៩ ឯកឧត្តម គួច ចំរើន អភិបាលខេត្តព្រះសីហនុ"
t4 = "This. 11,12 ២២.២២២.២២២,២២"
t5 = " ក "
print("kcc:", seg_kcc(t1))
print("kcc:", seg_kcc(t2))
print("kcc:", seg_kcc(t3))
print("kcc:", seg_kcc(t4))
print("kcc:", seg_kcc(t5))

kcc: ['យោ', 'ង', 'តា', 'ម', 'ប្រ', 'ភ', 'ព', 'ព័', 'ត៌', 'មា', 'ន', 'បា', 'ន', 'ឱ្យ', 'ដឹ', 'ង', 'ថា', ' ', 'កា', 'ល', 'ពី', 'ពេ', 'ល', 'ថ្មី', 'ៗ', 'នេះ', 'ក្រុ', 'ម', 'ច', 'ក្រ', 'ភ', 'ព', 'អ', 'ង់', 'គ្លេ', 'ស', ' ', 'Royal', ' ', 'Marines', ' ', 'ដែ', 'ល', 'មា', 'ន', 'មូ', 'ល', 'ដ្ឋា', 'ន', 'នៅ', ' ', 'Gibraltar', ' ', 'បា', 'ន', 'រឹ', 'ប', 'អូ', 'ស', 'យ', 'ក', 'នា', 'វា', 'ដឹ', 'ក', 'ប្រេ', 'ង', 'ឆៅ', 'រ', 'ប', 'ស់', 'អ៊ី', 'រ៉', 'ង់', 'ដែ', 'ល', 'ធ្វើ', 'ដំ', 'ណើ', 'រ', 'ទៅ', 'កា', 'ន់', 'រោ', 'ង', 'ច', 'ក្រ', 'ច', 'ម្រា', 'ញ់', 'ប្រេ', 'ង', 'នៅ', 'ក្នុ', 'ង', 'ប្រ', 'ទេ', 'ស', 'ស៊ី', 'រី', ' ', 'ដោ', 'យ', 'ក្រុ', 'ង', 'ឡុ', 'ង', 'ដ៍', 'អះ', 'អា', 'ង', 'ថា', ' ', 'កា', 'រ', 'រឹ', 'ប', 'អូ', 'ស', 'ត្រូ', 'វ', 'បា', 'ន', 'គេ', 'សំ', 'ដៅ', 'អ', 'នុ', 'វ', 'ត្ត', '៕']
kcc: ['ខែ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', ' ', 'N', '.', 'B', '.', ' ', 'ខែ', '?', ' ', 'Test', '?']
kcc: ['នៅ', 'រ', 'សៀ', 'ល', 'ថ្ងៃ', 'ទី', '២២', ' ', 'ខែ', ' ', 'ក', 'ក្ក', 'ដា', ' ', 'ឆ្នាំ', '២០១៩

In [12]:
def word2features(sent, i):
    word = sent[i]
    kccs = seg_kcc(word)

    features = {
        'bias': 1.0,
        'len(kccs)':len(kccs),
        'word.isdigit()': word.isdigit(),
        'word':word
    }

    for k in range(len(kccs)):
        features.update({f"kccs[{k}]":kccs[k]})

    if i > 0:
        word1 = sent[i-1]
        kccs1 = seg_kcc(word1)
        features.update({
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word': word1,
            '-1len(kccs)':len(kccs1),
        })
        for k in range(len(kccs1)):
            features.update({f"-1kccs1[{k}]":kccs1[k]})
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1]
        kccs1 = seg_kcc(word1)
        features.update({
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word': word1,
            '+1len(kccs)':len(kccs1),
        })
        for k in range(len(kccs1)):
            features.update({f"+1kccs1[{k}]":kccs1[k]})
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [15]:
X = []
y = []
bad_id = 0
for i in range(len(sentences)):
    sentence = sentences[i].split(" ")
    tags = pos_tags[i].split()
    if len(tags)!=len(sentence):
        bad_id += 1
        continue
    X.append(sent2features(sentence))
    y.append(tags)
print(bad_id, len(X))

0 20106


### Generate model inputs and split into training and test set

In [16]:
from sklearn.model_selection import train_test_split
indices = range(len(X))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1710) #0.20
print("len X_train:", len(X_train))
print("len y_train:", len(y_train))
print("len X_test:", len(X_test))
print("len y_test:", len(y_test))


len X_train: 16084
len y_train: 16084
len X_test: 4022
len y_test: 4022


In [17]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    # **rs.best_params_
    c1=0.1,           # 0.015, # not applicable for 'l2sgd'
    c2=0.05,          # 0.0037,
    verbose=True
)

In [18]:
crf.fit(X_train, y_train, X_dev=X_test, y_dev=y_test)

loading dev data to CRFsuite:   6%|▌         | 224/4022 [00:00<00:01, 2234.86it/s]

loading dev data to CRFsuite: 100%|██████████| 4022/4022 [00:01<00:00, 2344.11it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 273052
Seconds required: 1.926

L-BFGS optimization
c1: 0.100000
c2: 0.050000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=29.82 loss=2071959.60 active=270311 precision=0.002  recall=0.010  F1=0.003  Acc(item/seq)=0.168 0.000  feature_norm=1.00
Iter 2   time=17.57 loss=1817475.76 active=259370 precision=0.024  recall=0.023  F1=0.015  Acc(item/seq)=0.239 0.000  feature_norm=3.84
Iter 3   time=32.90 loss=1726719.27 active=262722 precision=0.020  recall=0.014  F1=0.009  Acc(item/seq)=0.206 0.000  feature_norm=3.81
Iter 4   time=17.82 loss=1659847.22 active=268177 precision=0.027  recall=0.024  F1=0.019  Acc(item/seq)=0.285 0.000  feature_norm=3.52
Iter 5   time=17.96 loss=1597871.11 

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.05, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=True)

In [20]:
import pickle
filename = f"sklearn_crf_pos_alt_{(crf.training_log_.__dict__['last_iteration']['item_accuracy_float']):.4f}.sav"

# save model to disk for further run
pickle.dump(crf, open(filename, 'wb'))
print("File:", filename)

File: sklearn_crf_pos_alt_0.8765.sav


In [21]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [22]:
result = loaded_model.score(X_train, y_train)
print("Train set num sentences:", len(y_train))
print("Performance on training set:", result)

Train set num sentences: 16084
Performance on training set: 0.9367140647852039


In [23]:
result = loaded_model.score(X_test, y_test)
print("Test set num sentences:", len(y_test))
print("Performance on test set:", result)


Test set num sentences: 4022
Performance on test set: 0.8765071373281915
